<a href="https://colab.research.google.com/github/hkl2005/MyDaSanYuan/blob/main/Dark%20Horse%20reborn%20pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# --- Install dependencies (Colab) ---
!pip install git+https://github.com/rongardF/tvdatafeed.git
!pip install pandas matplotlib ta

# --- Imports ---
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import ta
import matplotlib.pyplot as plt
import os

# --- Connect to TradingView ---
tv = TvDatafeed(username='', password='')  # guest mode

# --- Load tickers from TradingView watchlist file ---
# Example: upload your file via Colab sidebar → Files → then set path below
watchlist_file = "watchlist.txt"  # one ticker per line, e.g., OPPSTAR, GAMUDA
if os.path.exists(watchlist_file):
    with open(watchlist_file, "r") as f:
        tickers = [line.strip() for line in f if line.strip()]
else:
    # fallback if no file found
    tickers = ["OPPSTAR", "SCIENTX", "GAMUDA", "INARI", "DNEX"]

print(f"📂 Loaded {len(tickers)} tickers from watchlist")

# --- Output folder ---
os.makedirs("charts", exist_ok=True)

results = []

# --- Pattern check function ---
def check_pattern(df):
    try:
        df["EMA8"] = ta.trend.EMAIndicator(df["close"], window=8).ema_indicator()
        df["EMA21"] = ta.trend.EMAIndicator(df["close"], window=21).ema_indicator()
        df["EMA60"] = ta.trend.EMAIndicator(df["close"], window=60).ema_indicator()

        latest = df.iloc[-1]
        prev = df.iloc[-10] if len(df) > 10 else df.iloc[0]

        # Condition A: EMA alignment
        cond_alignment = latest["EMA8"] > latest["EMA21"] > latest["EMA60"]

        # Condition B: Sloping upwards
        cond_slope = (
            latest["EMA8"] > prev["EMA8"] and
            latest["EMA21"] > prev["EMA21"] and
            latest["EMA60"] > prev["EMA60"]
        )

        if cond_alignment and cond_slope:
            return "strong"
        elif cond_alignment:
            return "potential"
        else:
            return None
    except Exception:
        return None

# --- Scan tickers ---
for ticker in tickers:
    try:
        # Download ~1 year of daily data
        df = tv.get_hist(
            symbol=ticker,
            exchange="MYX",
            interval=Interval.in_daily,
            n_bars=300
        )

        if df is None or df.empty:
            print(f"⚠️ No data for {ticker}")
            continue

        result = check_pattern(df)
        if result in ["strong", "potential"]:  # ✅ capture both
            print(f"✅ {ticker} → {result}")

            # Save annotated chart
            plt.figure(figsize=(10,5))
            plt.plot(df.index, df["close"], label="Close", color="black")
            plt.plot(df.index, df["EMA8"], label="EMA8", color="green")
            plt.plot(df.index, df["EMA21"], label="EMA21", color="orange")
            plt.plot(df.index, df["EMA60"], label="EMA60", color="blue")
            plt.title(f"{ticker} ({result.upper()})")
            plt.legend()
            safe_ticker = ticker.replace(":", "_")
            plt.savefig(f"charts/{safe_ticker}_{result}.png")
            plt.close()

            results.append({
                "Ticker": ticker,
                "Last Price": df["close"].iloc[-1],
                "Match Type": result
            })

    except Exception as e:
        print(f"❌ Error with {ticker}: {e}")

# --- Export results ---
df_results = pd.DataFrame(results)
df_results.to_csv("KLSE_Oppstar_Patterns.csv", index=False)

print("🎉 Done.")
print("Strong matches:", len(df_results[df_results['Match Type']=='strong']))
print("Potential matches:", len(df_results[df_results['Match Type']=='potential']))
print("📂 Results saved to 'KLSE_Oppstar_Patterns.csv'")
print("📂 Charts saved in 'charts/' folder")




  Cloning https://github.com/rongardF/tvdatafeed.git to /tmp/pip-req-build-qkyk7jvy
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed.git /tmp/pip-req-build-qkyk7jvy
  Resolved https://github.com/rongardF/tvdatafeed.git to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


ERROR:tvDatafeed.main:error while signin


📂 Loaded 5 tickers from watchlist
✅ OPPSTAR → strong
✅ SCIENTX → strong
✅ INARI → strong
🎉 Done.
Strong matches: 3
Potential matches: 0
📂 Results saved to 'KLSE_Oppstar_Patterns.csv'
📂 Charts saved in 'charts/' folder
